In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory

# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# **READING THE DATA**

In [ ]:
ha=pd.read_csv('/kaggle/input/health-care-data-set-on-heart-attack-possibility/heart.csv')

In [ ]:
ha

Columns information

1) age
2) sex
3) chest pain type (4 values)
4) resting blood pressure
5) serum cholestoral in mg/dl
6) fasting blood sugar > 120 mg/dl
7) resting electrocardiographic results (values 0,1,2)
8) maximum heart rate achieved
9) exercise induced angina
10) oldpeak = ST depression induced by exercise relative to rest
11) the slope of the peak exercise ST segment
12) number of major vessels (0-3) colored by flourosopy
13) thal: 0 = normal; 1 = fixed defect; 2 = reversable defect
14) target: 0= less chance of heart attack 1= more chance of heart attack

In [ ]:
ha.info()

No missing value in dataset

# **DATA CLEANING**

In [ ]:
for i in ha:
    print(f'{i}={ha[i].unique()}')

12) number of major vessels (0-3) colored by flourosopy
There are only 0,1,2,3 vesses colored by flourosopy.. but in dataset ca=[0,2,1,3,4] so 4 must be removed from there...

In [ ]:
ha.loc[ha.ca==4]

In [ ]:
ha=ha.drop(index=ha.loc[ha.ca==4].index)

In [ ]:
ha=ha.reset_index(drop=True)

In [ ]:
ha

In [ ]:
hacopy=ha.copy()

Now we have a fully cleaned data

# ** Exploratory DATA Analysis **

now we have to analyze the 

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
from pandas_profiling import ProfileReport
x=ProfileReport(ha,explorative=True)
x.to_notebook_iframe()

From the above data profiling, the correlation matrix give a rough a idea about the dataset
1. As our target row in correlation has very less correlation with other features.
2. even there is not a very high correlation with each other. 
so we have to use nueral network for this datasets.

# Data preprocessing

In [ ]:
for i in ha:
    print(f'{i}={ha[i].unique()}')

There is some classification columns has more than 2 option so we have to change it under one hot encoding. 
* cp
* restecg
* slope
* ca
* thal

In [ ]:
ha=pd.get_dummies(data=ha,columns=['cp','restecg','slope','ca','thal'])

Now we have to scale the of some columns that is:
* age
* trestbps
* thalach
* chol
* oldpeak
'I am using a min max scalar for this'

In [ ]:
from sklearn.preprocessing import MinMaxScaler
a=MinMaxScaler()
ha[['age','trestbps','thalach','chol','oldpeak']]=a.fit_transform(ha[['age','trestbps','thalach','chol','oldpeak']])
ha

Now we have all our values in the between of 0 to 1
which is perfect for classification of data

# Prediction

In [ ]:
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras 

In [ ]:
# li=[]
# for i in range(30):
#     X_train,X_test,y_train,y_test=train_test_split(ha.drop(columns='target'),ha['target'],test_size=0.3,random_state=)
#     model=keras.Sequential([
#         keras.layers.Dense(20,input_shape=(len(X_train.columns),),activation='relu'),
#         keras.layers.Dense(10,activation='relu'),
#         keras.layers.Dense(1,activation='sigmoid')
#     ])
#     model.compile(optimizer='adam',
#                  loss='binary_crossentropy',
#                  metrics=['accuracy'])
#     model.fit(X_train,y_train,epochs=5)
#     li.append(model.evaluate(X_test,y_test))
# print(li)

In [ ]:
# o=[li[i][1] for i in range(len(li))]
# o.index(max(o))

In [ ]:
# max(o)

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(ha.drop(columns='target'),ha['target'],test_size=0.3,random_state=15)
model=keras.Sequential([
    keras.layers.Dense(20,input_shape=(len(X_train.columns),),activation='relu'),
    keras.layers.Dense(10,activation='relu'),
    keras.layers.Dense(1,activation='sigmoid')
])
model.compile(optimizer='adam',
             loss='binary_crossentropy',
             metrics=['accuracy'])
model.fit(X_train,y_train,epochs=50)
model.evaluate(X_test,y_test)

In [ ]:
l=model.predict(X_test)

In [ ]:
k=[]
for i in l:
    if i<=0.5:
        k.append(0)
    else:
        k.append(1)

In [ ]:
k[:10]

In [ ]:
y_test[0:10]

# Accuracy and Confusion matrix

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
print(classification_report(y_test,k))

In [ ]:
cm=tf.math.confusion_matrix(labels=y_test,predictions=k)
sns.heatmap(cm,annot=True,fmt='d')
plt.xlabel('predicted')
plt.ylabel('Truth')

***Working in progress, finding the way for more accuracy in model***